In [1]:
import cv2
import mediapipe as mp
import pyautogui
import time

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
cap = cv2.VideoCapture(0)

In [4]:
last_action_time = 0
cooldown = 0.25

In [5]:
with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
) as hands:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue

        # Flip and convert to RGB
        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process hands
        result = hands.process(frame_rgb)
        frame_height, frame_width, _ = frame.shape

        action = ""

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Get wrist (palm center)
                palm = hand_landmarks.landmark[0]
                x = int(palm.x * frame_width)
                y = int(palm.y * frame_height)

                # Visual feedback
                cv2.circle(frame, (x, y), 12, (255, 0, 255), -1)

                # Determine zones
                current_time = time.time()
                if current_time - last_action_time > cooldown:
                    if x < frame_width // 3:
                        pyautogui.press("left")
                        action = "⬅️ Move Left"
                        last_action_time = current_time
                    elif x > 2 * frame_width // 3:
                        pyautogui.press("right")
                        action = "➡️ Move Right"
                        last_action_time = current_time
                    elif y < frame_height // 3:
                        pyautogui.press("up")
                        action = "⬆️ Jump"
                        last_action_time = current_time
                    elif y > 2 * frame_height // 3:
                        pyautogui.press("down")
                        action = "⬇️ Slide"
                        last_action_time = current_time

        # Show action and hand
        if action:
            cv2.putText(frame, action, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)

        cv2.imshow("🖐️ Palm Gesture Controller", frame)

        # Exit on 'Esc'
        if cv2.waitKey(10) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()